尽管深度学习NLP的奇技淫巧很酷炫，但是基础的文本数据清洗却十分繁琐，因此还是需要整理相关的知识。文本数据中重要的信息就是时间，在本篇笔记中我将总结如何抽取和处理日期和时间数据。囊括的工具包括：python, pandas，bs, sql, excel等。

看到相关的代码我就会放进来，可能会有点乱，作为snippets使用吧。

# Python

## Regular Expression

- 正则表达式参考资料：https://www.py4e.com/html3/11-regex
- examples: https://cs.lmu.edu/~ray/notes/regex/
- Cheatsheet：
    - https://www.py4e.com/lectures3/Pythonlearn-11-Regex-Handout.txt
    - http://web.mit.edu/hackl/www/lab/turkshop/slides/regex-cheatsheet.pdf
    - https://www.runoob.com/python/python-reg-expressions.html
    - https://docs.python.org/zh-cn/3.7/library/re.html
- Python参考书籍：
    - http://do1.dr-chuck.com/pythonlearn/EN_us/pythonlearn.pdf

假设我们有许多纯文本格式的文件，我们需要从中识别和抽取出时间格式的数据，该如何操作呢？

## 批量读取文件

In [ ]:
import os

#文件路径
file_dir = "your path"
#找到文件路径下的所有文件名称，返回列表
file_list = os.listdir(file_dir)

for one_file in file_list:
    #重构文件路径
    file_path = os.path.join(file_dir,one_file)
    try:
        with open(file_path) as f:
            one_file = f.read()
        #有些文件可能损坏，无法读取
    except:
        print("cannot read" + one_file)

## 日期时间提取与格式化

参考资料：https://blog.csdn.net/qq_43404784/article/details/84704643
>在文本中日期呈现的方式各种各样：2018.12.2、2018.12.02、2018-12-2、2018-12-02、201/12/2、二零一八年十二月二日、2018年12月2日 等等

In [ ]:
# 利用正则表达式和apply函数将日期一次性提取出来：
import re
pattern = re.compile('\d{4}[\.\-/年]{1}\d{1,2}[\.\-/月]{1}\d{1,2}[日号]{0,1}|二.{3}年.{1,2}月.{1,3}[日号]{1}')
data['date'] = data['text'].apply(lambda s: pattern.findall(s))

In [14]:
# 参考资料：https://blog.csdn.net/mitongxue/article/details/103585450
import datetime
DATE =datetime.datetime.now()  
print(type(DATE)) #2019-12-17 18:29:05.494577
date=DATE.strftime("%Y/%m/%d")
print(date) #2019/12/17

<class 'datetime.datetime'>
2020/11/16


In [ ]:
#日期格式化
replace_dic = {'元': '01', '二○一七': '2017', '二○一八': '2018', '二〇一七': '2017', '二〇一八': '2018', '年': '-', '月': '-',
               '日': '','\.': '-', '/': '-', '号': '','三十一': '31', '三十': '30', '二十九': '29', '二十八': '28', '二十七': '27', 
               '二十六': '26', '二十五': '25','二十四': '24', '二十三': '23', '二十二': '22', '二十一': '21', '二十': '20', '十九': '19', 
               '十八': '18','十七': '17', '十六': '16', '十五': '15', '十四': '14', '十三': '13', '十二': '12', '十一': '11', '十': '10',
               '九': '09', '八': '08', '七': '07', '六': '06', '五': '05', '四': '04', '三': '03', '二': '02','一': '01'}

for key, value in replace_dic.items():
    data['date'] = data['date'].apply(lambda s: re.sub(key, value, s))

In [ ]:
# 自定义格式化函数。
def to_time(s):
    a = s.split('-')
    # return pd.datetime.strptime(s,'%Y-%m-%d') 
    return str(datetime.date(int(a[0]), int(a[1]), int(a[2])))

import datetime
data['date'] = data['date'].apply(to_time)

In [ ]:
# 或者在pandas操作
data['time'] = pd.to_datetime(data.date, format='%Y-%m-%d')

# Pandas

很多时候我们的数据是以表格形式存在，这时需要pandas批量提取或更改数据。pandas处理文本数据(pandas.Series.str)的常用方法(String methods)有：https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html


以下例子参考：https://www.dezyre.com/recipes/preprocess-string-data-within-pandas-dataframe

In [10]:
import pandas as pd
data = {'stringData': ['Sheldon 1 2019-09-29    3242.0',
                           'Copper 1 2020-12-25       3413.7',
                           'Raj 0 2014-05-25     2123.8',
                           'Howard 0 2017-09-24   1173.6',
                           'Leonard 1 2013-01-15    9134.0',
                           'Penny 0 2012-07-24    2755.6']}
df = pd.DataFrame(data, columns = ['stringData'])
df

,stringData
0,Sheldon 1 2019-09-29 3242.0
1,Copper 1 2020-12-25 3413.7
2,Raj 0 2014-05-25 2123.8
3,Howard 0 2017-09-24 1173.6
4,Leonard 1 2013-01-15 9134.0
5,Penny 0 2012-07-24 2755.6


In [11]:
df['state'] = df['stringData'].str.extract('([A-Z]\w{0,})', expand=True)
df['Boolean'] = df['stringData'].str.extract('(\d)', expand=True)
df['date'] = df['stringData'].str.extract('(....-..-..)', expand=True)
df['score'] = df['stringData'].str.extract('(\d\d\d\d\.\d)', expand=True)
df

,stringData,state,Boolean,date,score
0,Sheldon 1 2019-09-29 3242.0,Sheldon,1,2019-09-29,3242.0
1,Copper 1 2020-12-25 3413.7,Copper,1,2020-12-25,3413.7
2,Raj 0 2014-05-25 2123.8,Raj,0,2014-05-25,2123.8
3,Howard 0 2017-09-24 1173.6,Howard,0,2017-09-24,1173.6
4,Leonard 1 2013-01-15 9134.0,Leonard,1,2013-01-15,9134.0
5,Penny 0 2012-07-24 2755.6,Penny,0,2012-07-24,2755.6


In [ ]:
df['日期'] = df['时间'].str.extract(r'\d{4}年(\d{1,2}月\d{1,2}日)')
df['月份'] = df['日期'].str.extract(r'(\d{1,2})月')
df['年份'] = df['时间'].str.extract(r'(\d{4})年')

In [ ]:
# 区分不同数据格式
df2['时间'] = pd.to_datetime(df2['时间'], errors='coerce')
df_new = df[df2['讲者'].isna()]
# 添加新的columns
df_new['new'] = np.nan
# reset index
df_new.drop(df.columns[0], axis=1).reset_index(inplace=True)

## 多进程加快速度

In [ ]:
import multiprocessing
import time
def some_func(d):
    print(d)
data = [1, 2, ..., 10]
t1 = time.time()
pool = multiprocessing.Pool(processes = 3)
pool.map(some_func, data)
pool.close()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

# 网页文本提取

很多时候我们是抽取网页信息(html或json)。这部分待补充。